In [2]:
import feedparser

In [3]:
rss_meduza = "https://meduza.io/rss/all"
rss_vedomosti = "https://www.vedomosti.ru/rss/news"
rss_lenta = "https://lenta.ru/rss"

In [34]:
feed_meduza = feedparser.parse(rss_meduza)
feed_vedomosti = feedparser.parse(rss_vedomosti)
feed_lenta = feedparser.parse(rss_lenta)

In [5]:
meduza_articles = []
vedomosti_articles = []
lenta_articles = []

In [36]:
for item in feed_meduza['items']:
    if is_news(item):
        meduza_articles.append(get_text(item))

In [10]:
for item in feed_vedomosti['items']:
    vedomosti_articles.append(get_text(item))

In [11]:
for item in feed_lenta['items']:
    lenta_articles.append(get_text(item))

In [7]:
def is_news(item):
    if "news" in item['link']:
        return True
    else:
        return False

In [8]:
from newspaper import Article
def get_text(item):
    url = item['link']
    article = Article(url)
    article.download()
    article.parse()
    return article.text

In [37]:
len(meduza_articles)

35

### Preprocess articles

In [89]:
import nltk
#nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " "\
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

In [50]:
preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")
#> 'сказать видеть кто-то наступать грабли разочаровывать натравлять'

preprocess_text("По асфальту мимо цемента, Избегая зевак под аплодисменты. Обитатели спальных аррондисманов")
#> 'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

### Compare articles

In [16]:
from nltk.tokenize import word_tokenize, sent_tokenize

##data = "Mars is a cold desert world. It is half the size of Earth. "


In [98]:
print(sent_tokenize(data))
print(word_tokenize(data))

['Mars is a cold desert world.', 'It is half the size of Earth.']
['Mars', 'is', 'a', 'cold', 'desert', 'world', '.', 'It', 'is', 'half', 'the', 'size', 'of', 'Earth', '.']


In [17]:
file_docs = []

tokens = sent_tokenize(meduza_articles[0])
for line in tokens:
    file_docs.append(line)

print("Number of documents:",len(file_docs))

Number of documents: 12


In [112]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [43]:
print(meduza_articles[18])
print(vedomosti_articles[1])

Британские организации, работающие над созданием вакцины от COVID-19, подвергаются хакерским атакам со стороны других государств, в частности, Ирана, России и Китая. Об этом пишет The Guardian со ссылкой на данные Национального центра кибербезопасности.

Как отмечает издание, ни одна из атак пока не была успешной.

Во всем мире сейчас разрабатывается несколько десятков различных видов вакцин от COVID-19.

В Великобритании дальше всех в разработке вакцины продвинулись ученые из Института Дженнера в Оксфорде. Разрабатываемый там препарат уже запустили в производство параллельно с началом испытаний на людях — в случае, если они окажутся успешными, уже к сентябрю ученые рассчитывают получить до миллиона доз.
Британские университеты и научные организации, работающие над вакциной от коронавирусной инфекции, стали жертвами хакерских атак, пишет The Guardian со ссылкой на экспертов по кибербезопасности. «Считается, что за хакерскими атаками стоят государства, включая Иран и Россию, эксперты та

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(min_df=1)  

In [23]:
tfidf = vect.fit_transform(file_docs)

In [27]:
pairwise_similarity = tfidf * tfidf.T 
pairwise_similarity.toarray()

array([[1.        , 0.05879216, 0.        , 0.        , 0.05047745,
        0.        , 0.08639927, 0.0562797 , 0.        , 0.06356501,
        0.1270475 , 0.        ],
       [0.05879216, 1.        , 0.        , 0.        , 0.        ,
        0.06607413, 0.08264944, 0.        , 0.04307758, 0.03413778,
        0.06514273, 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.02974314,
        0.        , 0.        , 0.        , 0.07762103, 0.02399968,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.04096692,
        0.06614297, 0.        , 0.        , 0.        , 0.04236234,
        0.        , 0.        ],
       [0.05047745, 0.        , 0.02974314, 0.04096692, 1.        ,
        0.07209613, 0.        , 0.        , 0.        , 0.03603132,
        0.        , 0.        ],
       [0.        , 0.06607413, 0.        , 0.06614297, 0.07209613,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ,

In [52]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

##nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize)

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


print (cosine_sim(preprocess_text(meduza_articles[0]), preprocess_text(meduza_articles[0])))
print (cosine_sim(preprocess_text(meduza_articles[18]), preprocess_text(vedomosti_articles[1])))

1.0000000000000004
0.21626697905977543


In [55]:
preprocess_text(meduza_articles[18])

'британский организация работать создание вакцина covid 19 подвергаться хакерский атака сторона государство частность иран россия китай это писать the guardian ссылка данные национальный центр кибербезопасность отмечать издание атака пока успешный мир разрабатываться несколько десяток различный вид вакцина covid 19 великобритания далеко разработка вакцина продвигаться ученый институт дженнер оксфорд разрабатывать препарат запускать производство параллельно начало испытание человек  —  случай оказываться успешный сентябрь ученый рассчитывать получать миллион доза'

In [54]:
preprocess_text(vedomosti_articles[1])

'британский университет научный организация работать вакцина коронавирусный инфекция становиться жертва хакерский атака писать the guardian ссылка эксперт кибербезопасность  « считаться хакерский атака стоять государство включая иран россия эксперт также называть китай качество вероятный виновный », –  писать британский газета кибератака британский учреждение заниматься исследование новый коронавирус отслеживать россия иран также сообщать таблоид the mail on sunday'

In [115]:
import gensim
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

{'.': 0, 'коронавирус': 1, 'михаила': 2, 'мишустина': 3, 'нашли': 4, 'премьер-министра': 5, 'россии': 6, 'у': 7, ',': 8, '«': 9, '»': 10, 'владимиру': 11, 'доложил': 12, 'интерфакс': 13, 'мишустин': 14, 'об': 15, 'путину': 16, 'сам': 17, 'сообщает': 18, 'этом': 19, 'заявил': 20, 'на': 21, 'самоизоляцию': 22, 'уходит': 23, 'что': 24, 'андрея': 25, 'белоусова': 26, 'исполняющим': 27, 'назначить': 28, 'обязанности': 29, 'он': 30, 'предложил': 31, 'премьера': 32, ';': 33, 'был': 34, 'кандидатуру': 35, 'минут': 36, 'несколько': 37, 'поддержал': 38, 'подписан': 39, 'путин': 40, 'соответствующий': 41, 'уже': 42, 'указ': 43, 'через': 44, 'эту': 45}


In [116]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [119]:
tf_idf = gensim.models.TfidfModel(corpus)

### test

In [65]:
ved_preproc = [preprocess_text(item) for item in vedomosti_articles]

In [90]:
meduza_preproc = [preprocess_text(item) for item in meduza_articles]

In [91]:
print(ved_preproc[1])
print(meduza_preproc[18])

британский университет научный организация работать вакцина коронавирусный инфекция становиться жертва хакерский атака писать the guardian ссылка эксперт кибербезопасность  « считаться хакерский атака стоять государство включая иран россия эксперт также называть китай качество вероятный виновный », –  писать британский газета кибератака британский учреждение заниматься исследование новый коронавирус отслеживать россия иран также сообщать таблоид the mail on sunday
британский организация работать создание вакцина covid 19 подвергаться хакерский атака сторона государство частность иран россия китай это писать the guardian ссылка данные национальный центр кибербезопасность отмечать издание атака пока успешный мир разрабатываться несколько десяток различный вид вакцина covid 19 великобритания далеко разработка вакцина продвигаться ученый институт дженнер оксфорд разрабатывать препарат запускать производство параллельно начало испытание человек  —  случай оказываться успешный сентябрь учены

In [92]:
tagged_data

[TaggedDocument(words=['28', 'апрель', 'путин', 'продлять', 'режим', 'нерабочий', 'день', '11', 'май', 'объявлять', '6', '7', '8', 'май', 'официально', 'нерабочий', 'день', 'сохранение', 'заработный', 'плата', 'снимать', 'вводить', 'из-за', 'коронавирус', 'ограничение', 'регион', 'смочь', 'начинать', '12', 'май', 'постепенно', 'рекомендация', 'который', '5', 'май', 'должно', 'подготовить', 'правительство', 'сказать', 'путин'], tags=['0']),
 TaggedDocument(words=['британский', 'университет', 'научный', 'организация', 'работать', 'вакцина', 'коронавирусный', 'инфекция', 'становиться', 'жертва', 'хакерский', 'атака', 'писать', 'the', 'guardian', 'ссылка', 'эксперт', 'кибербезопасность', '«', 'считаться', 'хакерский', 'атака', 'стоять', 'государство', 'включая', 'иран', 'россия', 'эксперт', 'также', 'называть', 'китай', 'качество', 'вероятный', 'виновный', '»', ',', '–', 'писать', 'британский', 'газета', 'кибератака', 'британский', 'учреждение', 'заниматься', 'исследование', 'новый', 'коро

In [67]:
#def make_tagged(data):
#    return [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(data)]

labeled_questions=[]
labeled_questions.append(TaggedDocument(ved_preproc[i], df[df.index == i].qid1))
labeled_questions.append(TaggedDocument(questions2[i].split(), df[df.index == i].qid2))

In [70]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tagged_data = make_tagged(ved_preproc)
max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)
  
model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

iteration 0


/Users/yulialysenko/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19
iteration 20
iteration 21
iteration 22
iteration 23
iteration 24
iteration 25
iteration 26
iteration 27
iteration 28
iteration 29
iteration 30
iteration 31
iteration 32
iteration 33
iteration 34
iteration 35
iteration 36
iteration 37
iteration 38
iteration 39
iteration 40
iteration 41
iteration 42
iteration 43
iteration 44
iteration 45
iteration 46
iteration 47
iteration 48
iteration 49
iteration 50
iteration 51
iteration 52
iteration 53
iteration 54
iteration 55
iteration 56
iteration 57
iteration 58
iteration 59
iteration 60
iteration 61
iteration 62
iteration 63
iteration 64
iteration 65
iteration 66
iteration 67
iteration 68
iteration 69
iteration 70
iteration 71
iteration 72
iteration 73
iteration 74
iteration 75
iteration 76
iteration 77
iteratio

AttributeError: 'Doc2Vec' object has no attribute 'add'

In [93]:
model_doc2vec = Sequential()
model_doc2vec.add(Embedding(voacabulary_dim, 100, input_length=longest_document, weights=[training_weights], trainable=False))
model_doc2vec.add(LSTM(units=10, dropout=0.25, recurrent_dropout=0.25, return_sequences=True))
model_doc2vec.add(Flatten())
model_doc2vec.add(Dense(3, activation='softmax'))
model_doc2vec.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'Sequential' is not defined

In [75]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize(preprocess_text(meduza_articles[18]))
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)


V1_infer [-0.22701053  0.04783302  0.61104786 -0.11992019 -0.42534006 -0.43586242
  0.3560058  -0.10478023  0.02475094 -0.15430155 -0.13138168 -0.2817125
 -0.15292282  0.17810518 -0.05516502 -0.40829086 -0.4717365  -0.02208114
 -0.41543186  0.1934686 ]
[('196', 0.7340975999832153), ('183', 0.6604785919189453), ('123', 0.6554771661758423), ('171', 0.6537078619003296), ('173', 0.6532644629478455), ('199', 0.641342043876648), ('175', 0.6121119856834412), ('39', 0.6102547645568848), ('190', 0.6078641414642334), ('61', 0.6038334369659424)]


In [82]:
preprocess_text(vedomosti_articles[123])

'стоимость комплект оборудование мочь составлять около $ 10 000 ,  оценивать заместитель руководитель центр компетенция нти база  « сколтех »  виталий шуба . система мочь востребовать место массовый скопление человек ,  считать . целое рынок подобный решение россия мочь оцениваться сотня миллион доллар ,  полагать эксперт . мнение вице-президент  « ростелеком »  валерий ермаков ,  тепловизионный система оставаться востребовать пандемия коронавирус ,  позволять организация защищать сотрудник всплеск вирусный заболевание . \n'

In [77]:
preprocess_text(meduza_articles[18])

'британский организация ,  работать создание вакцина covid - 19 ,  подвергаться хакерский атака сторона государство ,  частность ,  иран ,  россия китай . это писать the guardian ссылка данные национальный центр кибербезопасность . \n \n отмечать издание ,  атака пока успешный . \n \n мир разрабатываться несколько десяток различный вид вакцина covid - 19 . \n \n великобритания далеко разработка вакцина продвигаться ученый институт дженнер оксфорд . разрабатывать препарат запускать производство параллельно начало испытание человек  —  случай ,  оказываться успешный ,  сентябрь ученый рассчитывать получать миллион доза . \n'